#### Names of people in the group

Please write the names of the people in your group in the next cell.

Nick Askari

Simen Peder Stang

In [ ]:
# We need to install 'ipython_unittest' to run unittests in a Jupyter notebook
!pip install -q ipython_unittest

You should consider upgrading via the '/local_disk0/.ephemeral_nfs/envs/pythonEnv-da4bef0c-b482-44e6-bd72-0cdc1f0962a7/bin/python -m pip install --upgrade pip' command.


In [ ]:
# Loading modules that we need
from pyspark.sql.dataframe import DataFrame
from collections import Counter
from pyspark.sql.functions import desc

In [ ]:
# A helper function to load a table (stored in Parquet format) from DBFS as a Spark DataFrame 
def load_df(table_name: "name of the table to load") -> DataFrame:
    return spark.read.parquet(table_name)

users_df = load_df("/user/hive/warehouse/users")
posts_df = load_df("/user/hive/warehouse/posts")

#### Subtask 1: implementing two functions
Implement these two functions:
1. 'compute_pearsons_r' that receives a DataFrame and two column names and returns the [Pearson correlation coefficient](https://en.wikipedia.org/wiki/Pearson_correlation_coefficient) between values of two columns;
2. 'make_tag_graph' that in the input receives the DataFrame containing the records related to 'questions' and returns a DataFrame with two columns 'u' and 'v'; the record for row i from the resulting DataFrame is a tuple (u_i, v_i). u_i and v_j are distinct tags and have appeared together for a question.

Please note that you should implement the 'compute_pearsons_r' yourself, so you should not use the 'DataFrame.stat.corr' method. Nevertheless, you can use 'DataFrame.stat.corr' to verify the correctness of your implementation.

In [ ]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import split, regexp_replace
from pyspark.sql.types import ArrayType, StringType
from pyspark.sql.functions import udf, explode, regexp_replace, split, col

def compute_pearsons_r(df: "a DataFrame", col1: "name of column A", col2: "name of column B") -> float:
    ## To-do!
    col1_data = df.select(col1).collect()
    col2_data = df.select(col2).collect()
    
    vals1 = []
    vals2 = []

    for row1, row2 in zip(col1_data, col2_data):
        if row1[col1] is not None and row2[col2] is not None:
            vals1.append(row1[col1])
            vals2.append(row2[col2])

    x_avg, y_avg = calc_average(vals1), calc_average(vals2)

    numerator = 0
    d_1, d_2 = 0, 0
    for x_i, y_i in zip(vals1, vals2):
        numerator += (x_i - x_avg) * (y_i - y_avg) 
        d_1 += (x_i - x_avg)**2
        d_2 += (y_i - y_avg)**2
    denominator = (d_1 * d_2) ** 0.5
        
    return numerator / denominator

# Helper function
def calc_average(values):
    sum = 0
    for val in values:
        sum += val
    return sum / len(values)

def make_tag_graph(df: "DataFrame containing question data") -> DataFrame:
    ## To-do!
    spark = SparkSession.builder.getOrCreate()

    all_tags = df.select('Tags').collect()
    vals = [row['Tags'] for row in all_tags]
    split_tags = [tag.strip('<>').split('><') for tag in vals]

    visited_tags = []
    dataframe_array = []
    for tags in split_tags:
        if len(tags) == 1:
            tag = tags[0]
            record = (tag, tag)
            dataframe_array.append(create_dict(record))
        else:
            all_perms = permutations(tags)
            for record in all_perms:
                dataframe_array.append(create_dict(record))

    return spark.createDataFrame(dataframe_array)
    

# Helper function
def create_dict(tupple):
    a, b = tupple[0], tupple[1]
    return {'u': a, 'v': b}

# Helper function
def permutations(array):
    permutations = []
    n = len(array)
    for i in range(n):
        for j in range(n):
            if i != j:  
                permutations.append((array[i], array[j]))
    return permutations



#### Subtask 2: implementing three functions
Impelment these three functions:
1. 'get_nodes' that, given the result from execution of 'make_tag_graph', returns a DataFrame with one column named 'id' that includes the tags that have appeared in the tag graph;
2. 'get_edges' that, given the result from execution of 'make_tag_graph', returns a DataFrame with two columns 'src' and 'dst' where 'src' is the source node and 'dst' is the destination node.


Note that the term 'tag graph' in this context refers to the DataFrame reuturned by executing 'make_tag_graph'. Furthermore, 'src' and 'dst' are distinct, so 'src' != 'dst'.

In [ ]:
def get_nodes(df: "DataFrame of the tag graph") -> DataFrame:
  ## To-do!
  spark = SparkSession.builder.getOrCreate()
  tags_u = df.select("u").rdd.flatMap(lambda x: x).collect()
  tags_v = df.select("v").rdd.flatMap(lambda x: x).collect()

  added_tags = []
  dataframe_array = []
  for tag in tags_u:
    if tag not in added_tags:
      added_tags.append(tag)
      dataframe_array.append({'id': tag})

  for tag in tags_v:
    if tag not in added_tags:
      added_tags.append(tag)
      dataframe_array.append({'id': tag})

  return spark.createDataFrame(dataframe_array)

def get_edges(df: "DataFrame of the tag graph") -> DataFrame:
  ## To-do!
  all_edges = df.withColumnRenamed("u", "src") \
              .withColumnRenamed("v", "dst") 

  filtered_edges = all_edges.filter(all_edges.src != all_edges.dst)

  return filtered_edges


In [ ]:
# Loading 'ipython_unittest' so we can use '%%unittest_main' magic command
%load_ext ipython_unittest

The ipython_unittest extension is already loaded. To reload it, use:
  %reload_ext ipython_unittest


#### Subtask 3: validating the implementation by running the tests

Run the cell below and make sure that all the tests run successfully.

In [ ]:
%%unittest_main
class TestTask3(unittest.TestCase):
  
  error_threshold = 0.03
  
  def test_corr1(self):
    # Pearson correlation coefficient between 'user reputation' and 'upvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "UpVotes")
    self.assertLessEqual(abs(result-0.5218138310114108), self.error_threshold)
    print(result)
  
  def test_corr2(self):
    # Pearson correlation coefficient between 'user reputation' and 'downvotes' received by users
    result = compute_pearsons_r(users_df, "Reputation", "DownVotes")
    self.assertLessEqual(abs(result-0.1473558141546844), self.error_threshold)
    print(result)

  def test_corr3(self):
    # Pearson correlation coefficient between 'question score' and the 'number of answers' it received
    result = compute_pearsons_r(posts_df[posts_df["PostTypeId"] == 1], "Score", "AnswerCount")
    self.assertLessEqual(abs(result-0.47855272641249674), self.error_threshold)
    print(result)
    
  def test_make_tag_graph(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    self.assertIsInstance(result, DataFrame)
    
    coulmn_names = Counter(map(str.lower, ['u', 'v']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, result.columns)), "Missing column(s) or column name mismatch")
    
    display(result)
    
    self.assertEqual(result.count(), 228830)
    
  def test_get_nodes(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    n = get_nodes(result)
    self.assertEqual(n.count(), 638)
    n.show()

  def test_get_edges(self):
    result = make_tag_graph(df=posts_df[posts_df["PostTypeId"] == 1])
    e = get_edges(result)
    
    coulmn_names = Counter(map(str.lower, ['src', 'dst']))
    self.assertCountEqual(coulmn_names, Counter(map(str.lower, e.columns)), "Missing column(s) or column name mismatch")
    
    self.assertEqual(e.count(), 225290)
    e.show()
    

0.5218138821617687
0.14735583044989373
0.47530986340467524
+----------------+----------------+
|             src|             dst|
+----------------+----------------+
|       education|     open-source|
|     open-source|       education|
|     data-mining|     definitions|
|     definitions|     data-mining|
|machine-learning|         bigdata|
|machine-learning|          libsvm|
|         bigdata|machine-learning|
|         bigdata|          libsvm|
|          libsvm|machine-learning|
|          libsvm|         bigdata|
|         bigdata|     scalability|
|         bigdata|      efficiency|
|         bigdata|     performance|
|     scalability|         bigdata|
|     scalability|      efficiency|
|     scalability|     performance|
|      efficiency|         bigdata|
|      efficiency|     scalability|
|      efficiency|     performance|
|     performance|         bigdata|
+----------------+----------------+
only showing top 20 rows

+----------------+
|              id|
+------------

u v machine-learning machine-learning education open-source open-source education data-mining definitions definitions data-mining databases databases machine-learning bigdata machine-learning libsvm bigdata machine-learning bigdata libsvm libsvm machine-learning libsvm bigdata bigdata scalability bigdata efficiency bigdata performance scalability bigdata scalability efficiency scalability performance efficiency bigdata efficiency scalability efficiency performance performance bigdata performance scalability performance efficiency nosql relational-dbms relational-dbms nosql data-mining clustering data-mining octave data-mining k-means data-mining categorical-data clustering data-mining clustering octave clustering k-means clustering categorical-data octave data-mining octave clustering octave k-means octave categorical-data k-means data-mining k-means clustering k-means octave k-means categorical-data categorical-data data-mining categorical-data clustering categorical-data octave categorical-data k-means data-mining clustering clustering data-mining algorithms algorithms nosql tools nosql processing nosql apache-hadoop tools nosql tools processing tools apache-hadoop processing nosql processing tools processing apache-hadoop apache-hadoop nosql apache-hadoop tools apache-hadoop processing bigdata r r bigdata r databases r efficiency r tools databases r databases efficiency databases tools efficiency r efficiency databases efficiency tools tools r tools databases tools efficiency r data-cleaning data-cleaning r apache-hadoop r r apache-hadoop machine-learning predictive-modeling predictive-modeling machine-learning processing processing bigdata statistics statistics bigdata statistics bigdata bigdata statistics bigdata tools bigdata data-stream-mining tools bigdata tools data-stream-mining data-stream-mining bigdata data-stream-mining tools databases nosql databases neo4j nosql databases nosql neo4j neo4j databases neo4j nosql definitions parallel definitions distributed parallel definitions parallel distributed distributed definitions distributed parallel clustering clustering bigdata google bigdata search google bigdata google search search bigdata search google recommender-system information-retrieval information-retrieval recommender-system nosql performance performance nosql clustering algorithms clustering similarity algorithms clustering algorithms similarity similarity clustering similarity algorithms tools data-stream-mining data-stream-mining tools bigdata performance bigdata databases bigdata nosql performance bigdata performance databases performance nosql databases bigdata databases performance databases nosql nosql bigdata nosql performance nosql databases data-mining machine-learning machine-learning data-mining machine-learning dimensionality-reduction machine-learning python dimensionality-reduction machine-learning dimensionality-reduction python python machine-learning python dimensionality-reduction nosql nosql nlp topic-model nlp lda topic-model nlp topic-model lda lda nlp lda topic-model nlp language-model language-model nlp feature-selection feature-extraction feature-selection dimensionality-reduction feature-extraction feature-selection feature-extraction dimensionality-reduction dimensionality-reduction feature-selection dimensionality-reduction feature-extraction scalability apache-hadoop scalability map-reduce scalability mongodb apache-hadoop scalability apache-hadoop map-reduce apache-hadoop mongodb map-reduce scalability map-reduce apache-hadoop map-reduce mongodb mongodb scalability mongodb apache-hadoop mongodb map-reduce bigdata efficiency bigdata performance efficiency bigdata efficiency performance performance bigdata performance efficiency nosql efficiency nosql indexing nosql data-indexing-techniques nosql .net efficiency nosql efficiency indexing efficiency data-indexing-techniques efficiency .net indexing nosql indexing efficiency indexing data-indexing-techniques indexing .net data-index

Success

......
----------------------------------------------------------------------
Ran 6 tests in 33.362s

OK
Out[77]: <unittest.runner.TextTestResult run=6 errors=0 failures=0>

#### Subtask 4: answering to questions about Spark related concepts

Please write a short description for the terms below---one to two short paragraphs for each term. Don't copy-paste; instead, write your own understanding.

1. What do the terms 'User-Defined Functions (UDFs)', 'Data Locality', 'Bucketing', 'Distributed Filesystem' mean in the context of Spark?

Write your descriptions in the next cell.

Your answers...

### User-Defined Functions (UDFs)
User defined functions is essentially if one wants to use their custom defined transformation on some data. In other words you can extend the capabilities of the functions already in spark, for something very spesific that might serve your purpose best. One normal usecase is forexample using a UDF within some following SQL statement. 

### Data Locality
This means that every time one performs some computations (for instance order an action) it is good if it is done on in the node in the cluster where the data resides. In other words, one wants to process data on the same node (or close) to where the actual data is stored. This helps improve performance. 

### Bucketing
Bucketing is an optimization technique in spark. You divide some data into multiple buckets, based on the values of some columns. When doing this you must spesify the columns and the number of columns. It is advantagous for certain queries, particullary if you want to perform join operations.

### Distributed Filesystem
A distributed filesystem is a storage that manages that across multiple machines, so you can pool many servers together to store data. This is what is meant by clusters, where clusters are just a group of machines whilst a node is a single machine.
